In [1]:
import spacy
# import de_ict_ner_model
import de_soski_ner_model
from spacy.language import Language
import logging
log = logging.getLogger(__name__)
import sys
from smart_open import open
import json


In [2]:
import os,glob
from tqdm import tqdm
import pandas as pd
import random
folder_path = '/mnt/home/kartikeysharma/sentence_extraction/test_quarterly_datasets'

## DATASET

In [3]:
import re
def get_all_jsonl_job_ad_files(folder_path):
    # folder_path = 'ads_annotated/'
    file_names_jsonl = []
    for filename in glob.glob(os.path.join(folder_path, '*.jsonl.bz2')):
        with open(filename, 'r') as f:
            file_names_jsonl.append(filename)

    date_regex = re.compile(r'\d{4}-\d{2}')

    # Extract the dates from the file names and create a list of tuples with the date and file name
    date_file_tuples = [(date_regex.search(name).group(0), name) for name in file_names_jsonl]

    # Sort the list of tuples based on the extracted dates
    sorted_tuples = sorted(date_file_tuples)

    # Create a list of sorted file names from the sorted tuples
    sorted_file_names = [tup[1] for tup in sorted_tuples]
    return sorted_file_names

In [4]:
file_names_jsonl = get_all_jsonl_job_ad_files(folder_path)
file_names_jsonl

['/mnt/home/kartikeysharma/sentence_extraction/test_quarterly_datasets/ads_raw-2023-03.jsonl.bz2']

In [5]:
dataset = '/mnt/home/kartikeysharma/sentence_extraction/quarterly_datasets/ads_raw-2023-03.jsonl.bz2'

In [6]:
@Language.component('paragrapher')
def set_sentence_starts(doc):
    for i, token in enumerate(doc):
        if i > 0 and doc[i-1].text.count("\n") > 1 :
            doc[i].is_sent_start = True
            # log.error(f"SENTENCE BOUNDARY: {i}")
    return doc


def has_softskill_C(sentence):
    for token in sentence:
        if token.ent_type_.endswith("_C"):
            return True

## Load the ner model

In [7]:
nlp = de_soski_ner_model.load()
senter = nlp.create_pipe("sentencizer")
nlp.add_pipe("sentencizer")

doc = nlp("This is the first 24.07.2021 sentence. This .is .the .second sentence.")
for sent in doc.sents:
    print(sent.text)

/mnt/home/kartikeysharma/sentence_extraction/softskill-completion/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This is the first 24.07.2021 sentence.
This .is .the .second sentence.


In [8]:
# from spacy.pipeline.senter import DEFAULT_SENTER_MODEL
# config = {"model": DEFAULT_SENTER_MODEL,}

nlp = de_soski_ner_model.load()
senter = nlp.create_pipe("sentencizer")
nlp.add_pipe("sentencizer")
nlp.add_pipe('paragrapher', after="transformer")

# Get the names of all components in the pipeline
pipeline_names = nlp.pipe_names
print("Pipeline component names:", pipeline_names)

# Print detailed information about each pipeline component
for name in pipeline_names:
    pipe_component = nlp.get_pipe(name)
    print(f"\nComponent name: {name}")
    print(f"Component type: {type(pipe_component)}")
    if hasattr(pipe_component, "labels"):
        print(f"Labels: {pipe_component.labels}")
    if hasattr(pipe_component, "vocab"):
        print(f"Vocab: {pipe_component.vocab}")
    if hasattr(pipe_component, "model"):
        print(f"Model: {pipe_component.model}")

Pipeline component names: ['transformer', 'paragrapher', 'ner', 'sentencizer']

Component name: transformer
Component type: <class 'spacy_transformers.pipeline_component.Transformer'>
Labels: ()
Vocab: <spacy.vocab.Vocab object at 0x7f62d3b53400>
Model: <spacy_transformers.layers.transformer_model.TransformerModel object at 0x7f62bf2d5d80>

Component name: paragrapher
Component type: <class 'function'>

Component name: ner
Component type: <class 'spacy.pipeline.ner.EntityRecognizer'>
Labels: ('SoftSkill', 'SoftSkill_C')
Vocab: <spacy.vocab.Vocab object at 0x7f62d3b53400>
Model: <thinc.model.Model object at 0x7f62bd2833c0>

Component name: sentencizer
Component type: <class 'spacy.pipeline.sentencizer.Sentencizer'>
Labels: ()


In [9]:
# Print the pipeline components in order
for i, component in enumerate(nlp.pipeline):
    print(f"Pipeline component {i}: {component[0]} ({type(component[1])})")

Pipeline component 0: transformer (<class 'spacy_transformers.pipeline_component.Transformer'>)
Pipeline component 1: paragrapher (<class 'function'>)
Pipeline component 2: ner (<class 'spacy.pipeline.ner.EntityRecognizer'>)
Pipeline component 3: sentencizer (<class 'spacy.pipeline.sentencizer.Sentencizer'>)


## process each sentence and prepare prompt 

In [10]:
def prepare_prompt(sentence):
    # nlp = de_soski_ner_model.load()
    # dc = nlp(sentence)
    # print(dc.ents)
    text = sentence.text
    words = text.split()
    num_tokens = len(words)

    for ent in sentence.ents:
        if ent.label_ == 'SoftSkill_C':
            text = text.replace(ent.text, f"<SoftSkill_C>{ent.text}</SoftSkill_C>")
        if ent.label_ == 'SoftSkill':
            text = text.replace(ent.text, f"<SoftSkill>{ent.text}</SoftSkill>")
    # print(sentence)
    return text, num_tokens

## extract softskill_c terms as dataframe from the single jsonL file

In [11]:
def get_softskill(data, num, language):
    # log.warning(self.args)
    # entities = []
    # entity_labels = []
    # enitit_start_char = []
    # entity_end_char = []
    entity_sent = []
    
    job_ids_list = []
    year_list = []
    quarter_list = []
    creation_date_list = []
    text_valid_list = [] 
    length_list = []
    language_list = []
    text_list = []
    tokens_in_sent_list = []
    pipeline_version_list = []
    actual_sentence_list = []

    with open(data) as infile:
        json_lines = infile.readlines()
        # random.seed(11)
        # random.shuffle(json_lines)

        for i,l in enumerate(tqdm(json_lines)):
            # print(i)
            if i > num-1:
                break
            data = json.loads(l)
            if data['language'] != language or data["text_valid"] != 1 or "adve_text_copy" not in data:
                continue
            
            doc = nlp(data["adve_text_copy"])
            # print(f'the value of doc is {doc}')

            job_ids = data["adve_iden_adve"]
            year = data["adve_time_year"]
            quarter = data["adve_time_quar"]
            # creation_date = data["creation_date"]
            text_valid = data["text_valid"]
            length = data["length"]
            language = data["language"]
            text = data["adve_text_copy"]
            pipeline_version = data["pipeline_version"]
            # print(f'entities are {doc.ents}')
            # print('************************  NEW JOB AD ***************************  NEW JOB AD *****************************')
            # print(f'value of doc.ents is : {doc.ents}')
            # print(f'value of doc.sents is : {doc.sents}')

            if doc.ents:
                for j, sent in enumerate(doc.sents):
                    # print(f'sentence nummer{j} is here --> {sent.text}')
                    if not has_softskill_C(sent):
                        continue
                    # print(f"-------------SENT {j}")
                    # print(f'The value of the {j}th sentence is *****  {sent.text}')

                    flag_softskill = 0
                    flag_softskill_C = 0
                    for token  in sent:
                        # print(f'the token is {token.text}, and its label is {token.ent_type_}')
                        if token.ent_type_ == "SoftSkill":
                            flag_softskill = 1
                        if token.ent_type_ == "SoftSkill_C":
                            flag_softskill_C = 1
                    if flag_softskill == 1 and flag_softskill_C == 1:
                        print(doc)
                        prompt_sent, num_tokens = prepare_prompt(sent)
                        actual_sentence = sent.text
                        # print(f'The value of the produced prompt is @@@@@@  {prompt_sent}')
                        actual_sentence_list.append(actual_sentence)
                        tokens_in_sent_list.append(num_tokens)
                        entity_sent.append(prompt_sent)
                        job_ids_list.append(job_ids)
                        year_list.append(year)
                        quarter_list.append(quarter)
                        # creation_date_list.append(creation_date)
                        text_valid_list.append(text_valid)
                        length_list.append(length)
                        language_list.append(language)
                        text_list.append(text)
                        pipeline_version_list.append(pipeline_version)

                        continue
    
    zipped_list = list(zip( job_ids_list, year_list, quarter_list,entity_sent,actual_sentence_list,tokens_in_sent_list, text_valid_list, length_list, language_list, pipeline_version_list, text_list))
    df = pd.DataFrame(zipped_list, columns =['job_ids_list', 'year_list', 'quarter_list','entity_sent', 'actual_sentence_list','tokens_in_sent_list', 'text_valid_list', 'length_list', 'language_list', 'pipeline_version_list', 'text_list']) 
    return df

In [26]:
df.iloc[4]

job_ids_list                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           sjmm_qua-2-02-2021-06-02853-1-000018488
year_list                                                                                                                                                

In [31]:
dataset = '/mnt/home/kartikeysharma/sentence_extraction/new_quarterly_datasets/ads_raw-2021-06.jsonl.bz2'
df = get_softskill(dataset, 25, 'de')
df

  0%|          | 25/39897 [00:07<3:07:26,  3.55it/s]


,job_ids_list,year_list,quarter_list,entity_sent,actual_sentence_list,tokens_in_sent_list,text_valid_list,length_list,language_list,pipeline_version_list,text_list
0,sjmm_qua-2-02-2021-06-02852-1-000018481,2021,2021-06-01,"Ihre Stärken• Matura/angefangenes Studium im Bereich Marketing • Erste Berufserfahrung im Bereich Marketing und Grundkenntnisse von Produktionsabläufen der Werbemittelproduktion von Vorteil• <SoftSkill>Interesse an klassischer Musik</SoftSkill> und <SoftSkill_C>weiteren Musikgenres•</SoftSkill_C> <SoftSkill>Freude am Verfassen zielgruppenspezifischer Texte•</SoftSkill> <SoftSkill_C>Selbständiges</SoftSkill_C>, <SoftSkill_C>zuverlässiges</SoftSkill_C> und <SoftSkill>exaktes Arbeiten•</SoftSkill> <SoftSkill>Organisationstalent•</SoftSkill> <SoftSkill>Kommunikatives Auftreten</SoftSkill>, <SoftSkill>Teamgeist•</SoftSkill> Professioneller Umgang mit Office-Programmen\n\n","Ihre Stärken• Matura/angefangenes Studium im Bereich Marketing • Erste Berufserfahrung im Bereich Marketing und Grundkenntnisse von Produktionsabläufen der Werbemittelproduktion von Vorteil• Interesse an klassischer Musik und weiteren Musikgenres• Freude am Verfassen zielgruppenspezifischer Texte• Selbständiges, zuverlässiges und exaktes Arbeiten• Organisationstalent• Kommunikatives Auftreten, Teamgeist• Professioneller Umgang mit Office-Programmen\n\n",46,1,2348,de,f5db770,"Die Tonhalle-Gesellschaft Zürich AG ist Betreiberin des Tonhalle-Orchesters Zürich und Veranstalterin diverser Konzert- und Veranstaltungsserien. Unter der Leitung seines Music Directors Paavo Järvi gehört das Tonhalle-Orchester Zürich zu den führenden Orchestern Europas. Es arbeitet mit international gefeierten Gastdirigent*innen, Solist*innen und Komponist*innen zusammen.In Ergänzung zu den zahlreichen Konzerten in Zürich geht das Orchester auf internationale Tourneen und trägt den Ruf Zürichs als lebendige Musikstadt in renommierte Musikzentren der Welt hinaus.\n\nDie Tonhalle-Gesellschaft Zürich AG sucht per Mitte August eine*n PRAKTIKANT*IN MARKETING (50?60%)\n\nIhr Verantwortungsbereich• Unterstützung bei Marketing-Aktivitäten (Erstellung und Distribution von Flyern, Plakaten, Anzeigen, Bannern etc.)• Administrative Unterstützung bei der Planung und Durchführung von Events (auch vor Ort, inkl. Abendeinsätze) • Administrative Unterstützung Inserate (Koordination, Akquise, Rechnungsstellung)• Organisation und Betreuung von Promotion-Massnahmen (Koordination Einsatzorte und Mitarbeitende)• Mitarbeit bei der Betreuung des Clubs TOZzukunft für junge Leute bis 40 Jahre (Organisation von Events, Kommunikations- massnahmen und Akquise-Aktionen)• Betreuung und Pflege von Partnerschaften• Betreuung CRM• Allgemeine administrative Aufgaben\n\nIhre Stärken• Matura/angefangenes Studium im Bereich Marketing • Erste Berufserfahrung im Bereich Marketing und Grundkenntnisse von Produktionsabläufen der Werbemittelproduktion von Vorteil• Interesse an klassischer Musik und weiteren Musikgenres• Freude am Verfassen zielgruppenspezifischer Texte• Selbständiges, zuverlässiges und exaktes Arbeiten• Organisationstalent• Kommunikatives Auftreten, Teamgeist• Professioneller Umgang mit Office-Programmen\n\nWir bieten• Eine vielseitige und verantwortungsvolle Herausforderung• Eine sehr selbständige Aufgabe• Offenes und hilfsbereites Team• Ein interessantes, lebendiges und vielfältiges Arbeitsumfeld mit Arbeitsplatz im Herzen von Zürich• Einblick in eine renommierte Kulturinstitution • Vergütung des Praktikums plus Freikarten für Konzerte des Tonhalle-Orchesters Zürich\n\nIhre Bewerbung mit den üblichen Unterlagen senden Sie bitte bis spätestens 30. Mai 2021 an:Michaela Braun, Leitung Marketing & Kommunikation; michaela.braun@tonhalle.c"
1,sjmm_qua-2-02-2021-06-02853-1-000018483,2021,2021-06-01,"Für unsere Gastronomie am Standort Baar-Inwil suchen wir per 1. Juli 2021 oder nach Vereinbarung eine <SoftSkill_C>freundliche</SoftSkill_C>, <SoftSkill_C>zuverlässige</SoftSkill_C> und <SoftSkill>aufmerks

In [29]:
# # with open('/mnt/home/kartikeysharma/sentence_extraction/new_quarterly_datasets/ads_raw-2022-03.jsonl.bz2') as infile:
# #     json_lines = infile.readlines()
# #     # random.seed(11)
# #     # random.shuffle(json_lines)
# #     num = 1
# #     for i,l in enumerate(tqdm(json_lines)):
# #         # print(i)
# #         if i > num-1:
# #             break
# #         data = json.loads(l)
# #         if data['language'] != 'de' or data["text_valid"] != 1 or "adve_text_copy" not in data:
# #             continue
        
# #         doc = nlp(data["adve_text_copy"])

# import spacy
# import json
# import pandas as pd
# from tqdm import tqdm


# # Sample DataFrame to store the results
# results = []

# # Path to your JSONL file
# jsonl_file_path = '/mnt/home/kartikeysharma/sentence_extraction/new_quarterly_datasets/ads_raw-2021-06.jsonl.bz2'

# # Number of documents to process (set to None to process all documents)
# num_documents = 14

# # Read JSON lines from the file and process each document
# with open(jsonl_file_path) as infile:
#     json_lines = infile.readlines()
#     for i, l in enumerate(tqdm(json_lines)):
#         if num_documents is not None and i >= num_documents:
#             break

#         data = json.loads(l)
#         if data['language'] != 'de' or data["text_valid"] != 1 or "adve_text_copy" not in data:
#             continue

#         # Process the document
#         doc = nlp(data["adve_text_copy"])

#         # Extract entities and labels from the processed document
#         entities = [(ent.text, ent.label_) for ent in doc.ents]

#         # Add the information to the results list
#         results.append({
#             "Document ID": i + 1,
#             "Text": data["adve_text_copy"],
#             "Entities": entities
#         })

# # Create a DataFrame from the results list
# df2 = pd.DataFrame(results)
# df2


In [28]:
df2[df2['Document ID'] == 12]

,Document ID,Text,Entities
11,12,"Für unsere Abteilung Handwerk & Kunst, an unserem Standort Baar-Inwil, suchen wir per\n1. August 2021 für 6 Monate zwei engagierte Personen für ein\nPraktikum 100%\nUntere Rainstrasse 31, 6340 Baar\nAufgaben\n\n\tUnterstützen und Begleiten einer Gruppe von erwachsenen Menschen mit einer psychischen, geistigen und/oder körperlichen Beeinträchtigung\n\tMithelfen beim Herstellen von marktfähigen Produkten mit verschiedensten Materialien wie Papier, Holz, Ton, Textil\n\tMithelfen beim Bearbeiten von Kundenaufträgen mit und durch die Klientinnen und Klienten\n\nProfil\n\n\tAbgeschlossene Berufsausbildung oder Matura\n\tKreatives und handwerkliches Geschick\n\tFlair im Umgang mit Menschen mit einer Beeinträchtigung\n\tGute PC-Anwenderkenntnisse (MS Office)\n\nKontakt\nCéline Cudré\nSachbearbeiterin HR\n041 781 64 99","[(engagierte, SoftSkill), (Kreatives, SoftSkill_C), (handwerkliches Geschick, SoftSkill), (Flair im Umgang mit Menschen mit einer Beeinträchtigung, SoftSkill)]"


In [ ]:
data = '/mnt/home/kartikeysharma/sentence_extraction/new_quarterly_datasets/ads_raw-2023-03.jsonl.bz2'
num = 2
language = 'de'
entity_sent = []
job_ids_list = []
year_list = []
quarter_list = []
creation_date_list = []
text_valid_list = [] 
length_list = []
language_list = []
text_list = []
tokens_in_sent_list = []
pipeline_version_list = []
actual_sentence_list = []

with open(data) as infile:
    json_lines = infile.readlines()
    # random.seed(11)
    # random.shuffle(json_lines)

    for i,l in enumerate(tqdm(json_lines)):
        # print(i)
        if i > num-1:
            break
        data = json.loads(l)
        if data['language'] != language or data["text_valid"] != 1 or "adve_text_copy" not in data:
            continue
        
        doc = nlp(data["adve_text_copy"])
        # print(f'the value of doc is {doc}')

        job_ids = data["adve_iden_adve"]
        year = data["adve_time_year"]
        quarter = data["adve_time_quar"]
        # creation_date = data["creation_date"]
        text_valid = data["text_valid"]
        length = data["length"]
        language = data["language"]
        text = data["adve_text_copy"]
        pipeline_version = data["pipeline_version"]
        # print(f'entities are {doc.ents}')
        # print('************************  NEW JOB AD ***************************  NEW JOB AD *****************************')
        # print(f'value of doc.ents is : {doc.ents}')
        # print(f'value of doc.sents is : {doc.sents}')

        if doc.ents:
            for j, sent in enumerate(doc.sents):
                # print(f'sentence nummer{j} is here --> {sent.text}')
                if not has_softskill_C(sent):
                    continue
                # print(f"-------------SENT {j}")
                # print(f'The value of the {j}th sentence is *****  {sent.text}')

                flag_softskill = 0
                flag_softskill_C = 0
                for token  in sent:
                    # print(f'the token is {token.text}, and its label is {token.ent_type_}')
                    if token.ent_type_ == "SoftSkill":
                        flag_softskill = 1
                    if token.ent_type_ == "SoftSkill_C":
                        flag_softskill_C = 1
                if flag_softskill == 1 and flag_softskill_C == 1:
                    prompt_sent, num_tokens = prepare_prompt(sent)
                    actual_sentence = sent.text
                    # print(f'The value of the produced prompt is @@@@@@  {prompt_sent}')
                    actual_sentence_list.append(actual_sentence)
                    tokens_in_sent_list.append(num_tokens)
                    entity_sent.append(prompt_sent)
                    job_ids_list.append(job_ids)
                    year_list.append(year)
                    quarter_list.append(quarter)
                    # creation_date_list.append(creation_date)
                    text_valid_list.append(text_valid)
                    length_list.append(length)
                    language_list.append(language)
                    text_list.append(text)
                    pipeline_version_list.append(pipeline_version)

                    continue

zipped_list = list(zip( job_ids_list, year_list, quarter_list,entity_sent,actual_sentence_list,tokens_in_sent_list, text_valid_list, length_list, language_list, pipeline_version_list, text_list))
df = pd.DataFrame(zipped_list, columns =['job_ids_list', 'year_list', 'quarter_list','entity_sent', 'actual_sentence_list','tokens_in_sent_list', 'text_valid_list', 'length_list', 'language_list', 'pipeline_version_list', 'text_list']) 

In [22]:
import pandas as pd
# df.style.set_properties(max_colwidth=100)
pd.set_option('display.max_colwidth', 1800)



# Your pandas code goes here...
# Instead of just calling the dataframe, use the display() function
df


,job_ids_list,year_list,quarter_list,entity_sent,actual_sentence_list,tokens_in_sent_list,text_valid_list,length_list,language_list,pipeline_version_list,text_list
0,sjmm_qua-2-04-2023-03-05023-1-000188850,2023,1,"optimalerweise mehrjährige Berufserfahrung in Montagetätigkeit\n• <SoftSkill>Affinität zu Mechanik</SoftSkill>, <SoftSkill>gutes Gespür für Prozessabläufe</SoftSkill>\n• Lesen von technischen Zeichnungen und Stücklisten\n• Gute Deutschkenntnisse in Wort und Schrift, weitere Fremdsprachen von Vorteil\n• <SoftSkill_C>Freundliches</SoftSkill_C> und <SoftSkill>gepflegtes Auftreten</SoftSkill>\n• <SoftSkill>Freude an der Arbeit mit innovativen Technologien</SoftSkill>\n• <SoftSkill>Teamfähigkeit</SoftSkill>, <SoftSkill>Zuverlässigkeit</SoftSkill>, <SoftSkill>Ehrlichkeit</SoftSkill> und <SoftSkill>Effizienz</SoftSkill>, <SoftSkill>genaue Arbeitsweise</SoftSkill> sowie <SoftSkill>Hands-onMentalität</SoftSkill>\nWIR BIETEN DIR:\n• Einen topmodernen und zukunftsorientierten Arbeitsplatz in einem expandierenden Unternehmen\nin der Herstellung von Produktionslinien für Etikettiersysteme\n• Möglichkeit, dich kreativ einzubringen und attraktive Entwicklungsmöglichkeiten in einem\nvielseitigen Aufgabengebiet\n• Faire Bezahlung, 25 Tage Ferien, gratis Parkplatz und weitere Mitarbeiter-Benefits\n• Ein gut eingespieltes, motiviertes und kompetentes Team, in dem man sich gegenseitig unterstützt\n• Eigentümer, die selbst Hand anlegen und einen Firmenstandort mitten im Grünen, aber nur 20\nMinuten von Winterthur entfernt\nWenn du dich in der Beschreibung wiederfindest und neugierig bist auf mehr von unserem BLUMER-Team\nund der Erfolgsgeschichte, schick uns doch deinen Lebenslauf mit Foto und deine Zeugnisse oder den Link\nzu deinem LinkedIn- oder Xing-Account via Email an Barbara Wucsits auf [email].","optimalerweise mehrjährige Berufserfahrung in Montagetätigkeit\n• Affinität zu Mechanik, gutes Gespür für Prozessabläufe\n• Lesen von technischen Zeichnungen und Stücklisten\n• Gute Deutschkenntnisse in Wort und Schrift, weitere Fremdsprachen von Vorteil\n• Freundliches und gepflegtes Auftreten\n• Freude an der Arbeit mit innovativen Technologien\n• Teamfähigkeit, Zuverlässigkeit, Ehrlichkeit und Effizienz, genaue Arbeitsweise sowie Hands-onMentalität\nWIR BIETEN DIR:\n• Einen topmodernen und zukunftsorientierten Arbeitsplatz in einem expandierenden Unternehmen\nin der Herstellung von Produktionslinien für Etikettiersysteme\n• Möglichkeit, dich kreativ einzubringen und attraktive Entwicklungsmöglichkeiten in einem\nvielseitigen Aufgabengebiet\n• Faire Bezahlung, 25 Tage Ferien, gratis Parkplatz und weitere Mitarbeiter-Benefits\n• Ein gut eingespieltes, motiviertes und kompetentes Team, in dem man sich gegenseitig unterstützt\n• Eigentümer, die selbst Hand anlegen und einen Firmenstandort mitten im Grünen, aber nur 20\nMinuten von Winterthur entfernt\nWenn du dich in der Beschreibung wiederfindest und neugierig bist auf mehr von unserem BLUMER-Team\nund der Erfolgsgeschichte, schick uns doch deinen Lebenslauf mit Foto und deine Zeugnisse oder den Link\nzu deinem LinkedIn- oder Xing-Account via Email an Barbara Wucsits auf [email].",173,1,2249,de,bb0e34e,"Wir sind ein internationales Unternehmen, tätig in Entwicklung, Fabrikation, Vertrieb und Unterhalt von\nMaschinen und Produktionsanlagen für die Weiterverarbeitung nach dem Druck, sowie im Handel von\nVerbrauchsmaterialien für die grafische Branche und die Verpackungsindustrie.\nZur Verstärkung am Standort Oberneunforn suchen wir per sofort\noder nach Vereinbarung eine/n engagierte/n, selbstständige/n\nMASCHINEN-/ POLYMECHANIKER/IN\n FACHRICHTUNG MONTAGE\n 80-100%\nDEINE AUFGABEN:\n• Selbstständige mechanische Montage von unseren Maschinenmodulen und Etikettiermaschinen\n• Funktionskontrolle und Erstinbetriebnahme im Werk\n• Sporadische Service-Einsätze für Wartungs- und Reparaturarbeiten\nDU BRINGST MIT:\n• Abgeschlossene Berufslehre als Maschinen- oder Polymechaniker mit Fachrichtung M

## create complete dataset across all years

In [38]:
import pandas as pd
import os

def get_complete_df(folder_path, num, lang, batch_size=5):
    list_file_names = get_all_jsonl_job_ad_files(folder_path)
    batches = [list_file_names[i:i+batch_size] for i in range(0, len(list_file_names), batch_size)]
    
    for i, batch in enumerate(batches):
        master_GS = []
        for jsonl_file in batch:
            # print(jsonl_file)
            child_GS_df = get_softskill(jsonl_file, num, lang)
            
            master_GS.append(child_GS_df)

        master_df = pd.concat(master_GS)
        # Save dataframe as a CSV file
        csv_file_name = f"batch_{i+1}.csv"
        master_df.to_csv(os.path.join(folder_path, csv_file_name), index=False)



In [37]:
get_complete_df(folder_path, 1, 'de', batch_size=10)

/mnt/home/kartikeysharma/sentence_extraction/quarterly_datasets/ads_raw-2017-03.jsonl.bz2


  0%|          | 1/6455 [00:00<1:35:57,  1.12it/s]


/mnt/home/kartikeysharma/sentence_extraction/quarterly_datasets/ads_raw-2018-06.jsonl.bz2


  0%|          | 1/5353 [00:00<59:31,  1.50it/s]


/mnt/home/kartikeysharma/sentence_extraction/quarterly_datasets/ads_raw-2016-06.jsonl.bz2


...

  0%|          | 1/5366 [00:00<39:15,  2.28it/s]


/mnt/home/kartikeysharma/sentence_extraction/quarterly_datasets/ads_raw-2015-06.jsonl.bz2


  0%|          | 1/4939 [00:00<34:54,  2.36it/s]


/mnt/home/kartikeysharma/sentence_extraction/quarterly_datasets/ads_raw-2015-12.jsonl.bz2


  0%|          | 1/4889 [00:00<44:37,  1.83it/s]


In [ ]:
# import os
# import pandas as pd

# def get_complete_df(folder_path, num, lang):

#     list_file_names = get_all_jsonl_job_ad_files(folder_path)
#     master_GS = []
#     for jsonl_file in list_file_names:
#         child_GS_df = get_softskill(jsonl_file, num, lang)
#         master_GS.append(child_GS_df)

#     master_df = pd.concat(master_GS)
#     return master_df